In [1]:
#!pip install --quiet optuna
import optuna
import torch
from torch import optim
from Model import resnet18_model_fine_tuning
import DataLoader

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Params to learn:
	 fc.weight
	 fc.bias


In [2]:
# device - cpu or gpu?
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
batch_size = 128
epochs = 10
log_interval = 10
n_train_examples = batch_size * 30
n_valid_examples = batch_size * 10

def get_model():
  return resnet18_model_fine_tuning

def objective(trial):
  # Generate the model.
  model = get_model() # Already on device
  
  # Generate the optimizers.
  lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True) # log=True, will use log scale to interplolate between lr
  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
  
  # Get Data loader
  train_dataset = DataLoader.VoxCaleb1MelSpecDataset('./drive/MyDrive/VoxCeleb1', 'train', input_size=224, download=False)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=2)
  valid_dataset = DataLoader.VoxCaleb1MelSpecDataset('./drive/MyDrive/VoxCeleb1', 'val', input_size=224, download=False)
  valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=2)
  # Training of the model.
  for epoch in range(epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
      # Limiting training data for faster epochs.
      if batch_idx * batch_size >= n_train_examples:
        break
      
      data, target = data.view(data.size(0), -1).to(device), target.to(device)
      
      optimizer.zero_grad()
      output = model(data)
      loss = F.nll_loss(output, target)
      loss.backward()
      optimizer.step()
    
    # Validation of the model.
    model.eval()
    correct = 0
    with torch.no_grad():
      for batch_idx, (data, target) in enumerate(valid_loader):
        # Limiting validation data.
        if batch_idx * batch_size >= n_valid_examples:
          break
        data, target = data.view(data.size(0), -1).to(device), target.to(device)
        output = model(data)
        # Get the index of the max log-probability.
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
      
    accuracy = correct / min(len(valid_loader.dataset), n_valid_examples)
    
    # report back to Optuna how far it is (epoch-wise) into the trial and how well it is doing (accuracy)
    trial.report(accuracy, epoch)
    # then, Optuna can decide if the trial should be pruned
    # Handle pruning based on the intermediate value.
    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()
      
  return accuracy


In [4]:
# now we can run the experiment
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(study_name="mnist-fc", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100, timeout=600)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
print("Study statistics: ")
print(" Number of finished trials: ", len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")

for key, value in trial.params.items():
 print(" {}: {}".format(key, value))

[I 2023-01-13 22:22:00,587] A new study created in memory with name: mnist-fc
[W 2023-01-13 22:22:10,106] Trial 0 failed because of the following error: RuntimeError('Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [128, 150528]')
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-3-f2c4ac6a27fe>", line 35, in objective
    output = model(data)
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torchvision/models/resnet.py", line 285, in forward
    return self._forward_impl(x)
  File "/usr/local/lib/python3.8/dist-packages/torchvision/models/resnet.py", line 268, in _forward_impl
    x = self.conv1(x)
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py", lin

RuntimeError: ignored